# Line Chat Bot Record Expenses
##### Mr.Jakkrit Sonsanit

***

### Import Library 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from firebase import firebase

### Create New DataFrame

In [2]:
# dfe = pd.DataFrame({
#     'date':[],
#     'text':[],
#     'money':[],
#     'cate':[]
# })

## EXPENSES
***

### Load Dataframe

In [3]:
dfe = pd.read_csv('Expenses.csv')
dfe.tail()

,date,text,money,cate
115,2019-10-22,ค่าสองแถว,8.0,Transportation
116,2019-10-22,ค่าแท๊กซี่,12.0,Transportation
117,2019-10-22,ค่าแท๊กซี่,15.0,Transportation
118,2019-10-22,ค่า2แถว,10.0,Transportation
119,2019-10-22,ก๋วยเตี๋ยว,30.0,Food


### Connect Firebase

In [4]:
authentication = firebase.FirebaseAuthentication('wI4pPnsc1m2Ei7E3Q3drORPWtDBo927IxwBsA6ZH', 'palm8ikaa2@gmail.com', extra={'id': 123})
firebase = firebase.FirebaseApplication('https://line-bot-chrins.firebaseio.com/', authentication=authentication)

### Query Expenses from firebase

In [5]:
result = firebase.get('/expenses', None)

### Append new expenses in dataframe

In [6]:
tmp = list()
old_numlist = len(dfe)
for k, v in result.items():
    tmp.append(v)
for i in range(len(tmp)):
    if i >= old_numlist:
        dfe = dfe.append(tmp[i], ignore_index=True)
dfe.date = pd.to_datetime(dfe.date)
dfe.tail()

,date,text,money,cate
125,2019-10-24,น้ำมะตูม,5.0,NaN
126,2019-10-24,ชาหวาน,20.0,NaN
127,2019-10-24,ข้าวเย็น,35.0,NaN
128,2019-10-24,น้ำเต้าหู้,10.0,NaN
129,2019-10-25,ซื้อเกม,190.0,NaN


### Append expenses on day in dataframe

In [202]:
# for k, v in result.items():
#     if v['date'] == '6/9/2019':
#         dfe = dfe.append(v, ignore_index=True)
# dfe.date = pd.to_datetime(dfe.date)
# dfe.head()

### Append all expenses to dataframe

In [203]:
# for k, v in result.items():
#     dfe = dfe.append(v, ignore_index=True)
# dfe.date = pd.to_datetime(dfe.date)
# dfe.head()

### Set NaN Category is Food

In [7]:
dfe.cate = dfe.cate.fillna('Food')
dfe.tail(10)

,date,text,money,cate
120,2019-10-23,ปลากระป๋อง,18.0,Food
121,2019-10-23,อิชิตัน,25.0,Food
122,2019-10-23,กับข้าว,50.0,Food
123,2019-10-24,นม,10.0,Food
124,2019-10-24,ข้าวเที่ยง,35.0,Food
125,2019-10-24,น้ำมะตูม,5.0,Food
126,2019-10-24,ชาหวาน,20.0,Food
127,2019-10-24,ข้าวเย็น,35.0,Food
128,2019-10-24,น้ำเต้าหู้,10.0,Food
129,2019-10-25,ซื้อเกม,190.0,Food


### Change category

In [8]:
t = ['Entertainment', 'Transportation', 'Shopping', 'Food', 'Bill', 'Other']
n = 129
dfe.at[n, 'cate'] = t[0]
dfe.iloc[n:]

,date,text,money,cate
129,2019-10-25,ซื้อเกม,190.0,Entertainment


### Save to csv

In [9]:
dfe.to_csv('Expenses.csv', index=False)

## INCOME
***

### Create DataFrame

In [10]:
# dfi = pd.DataFrame({
#     'date':[],
#     'text':[],
#     'money':[],
# })

### Load DataFrame

In [11]:
dfi = pd.read_csv('income.csv')

### Query income from firebase

In [12]:
result = firebase.get('/income', None)

### Append new income 

In [13]:
tmp = list()
old_numlist = len(dfi)
for k, v in result.items():
    tmp.append(v)
for i in range(len(tmp)):
    if i >= old_numlist:
        dfi = dfi.append(tmp[i], ignore_index=True)
dfi.date = pd.to_datetime(dfi.date)
dfi.tail()

,date,text,money
0,2019-10-01,พ่อให้,14000.0
1,2019-10-17,ได้เงิน,625.0
2,2019-10-18,ได้เงิน,155.0


### Save to csv

In [14]:
dfi.to_csv('income.csv', index=False)

## Summary 
***

### Balance

In [15]:
print("Balance : %.2f Bath" %(sum(dfi.money)-sum(dfe.money)))

Balance : 1259.23 Bath


### Total Incom and Expenses 

In [16]:
print("Total Income :", sum(dfi.money), 'Bath')
print("Total Expenses :", sum(dfe.money), 'Baht')

Total Income : 14780.0 Bath
Total Expenses : 13520.77 Baht


### Total Expenses in day

In [17]:
d = "2019-"+input("mm-dd :")
print("Total Expenses in", d, ":", sum(dfe[dfe.date == d]['money']), 'Baht')

mm-dd :10-21
Total Expenses in 2019-10-21 : 220.0 Baht


### Total Expenses in each month 

In [19]:
tmp = dfe.rename(columns={"date": "Month", "money":"Money"})
tmp.groupby(tmp.Month.dt.month).sum()

,Money
Month,
10,12937.77


### Total Expenses in each category

In [19]:
dfe.groupby(['cate']).sum()

,money
cate,
Bill,6457.77
Entertainment,1740.00
Food,3286.00
Other,210.00
Saving,20.00
Shopping,1104.00
Transportation,305.00


### Expenses in each category

In [20]:
ls = ['1.Food', '2.Entertainment', '3.Bill', '4.Saving', '5.Shopping', '6.Transportation', '7.Other']
print(*ls)
dfe[dfe.cate == ls[int(input('Enter number of category '))-1][2:]]

1.Food 2.Entertainment 3.Bill 4.Saving 5.Shopping 6.Transportation 7.Other
Enter number of category 2


,date,text,money,cate
54,2019-10-09,ค่าหนัง,140.0,Entertainment
55,2019-10-10,เติมเกม,300.0,Entertainment
56,2019-10-10,ดูหนัง,100.0,Entertainment
59,2019-10-11,ค่าเหล้า,120.0,Entertainment
81,2019-10-16,ค่าเหล้า,330.0,Entertainment
90,2019-10-18,ค่าเหล้า,260.0,Entertainment
93,2019-10-19,นวด,450.0,Entertainment
99,2019-10-20,ธูปเทียน,20.0,Entertainment
101,2019-10-20,ธูปเทียน,20.0,Entertainment


### Total in each day

In [36]:
dfe.groupby(dfe.date.dt.day).sum()

,money
date,
1,1026.77
2,200.00
3,5630.00
4,280.00
5,260.00
6,494.00
7,51.00
8,280.00
9,425.00
